In [3]:
%defaultDatasource jdbc:h2:mem:db

# Analisando o Uso de Medicamentos
* Esta tarefa analisará os dados conforme três abordagens: análise do comportamento estatístico; análise de correlação entre pares; análise de correlação de variáveis.
* A partir dos arquivos CSV que estão no diretórios /data/nhanes considere as seguintes tabelas SQL.

In [4]:
CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade
FROM CSVREAD('../nhanes/demographic-person.csv');

CREATE TABLE Medicamento (
  Codigo VARCHAR(6) NOT NULL,
  NomeMedicamento VARCHAR(100) NOT NULL,
  PRIMARY KEY(Codigo)
) AS SELECT
    Codigo,
    NomeMedicamento
FROM CSVREAD('../nhanes/medications-drug.csv');

CREATE TABLE UsoMedicamento (
  IdPessoa VARCHAR(5) NOT NULL,
  CodMedicamento VARCHAR(6) NOT NULL,
  DiasUso INTEGER,
  FOREIGN KEY(IdPessoa)
    REFERENCES Pessoa(Id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(CodMedicamento)
    REFERENCES Medicamento(Codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    IdPessoa,
    CodMedicamento,
    DiasUso
FROM CSVREAD('../nhanes/medications-use.csv');

In [ ]:
SELECT * FROM Pessoa;
SELECT * FROM Medicamento;
SELECT * FROM UsoMedicamento;

# I. Análise de comportamento estatístico

## 1) Contabilizando o uso de medicamentos

* Liste o nome de todos os medicamentos e a quantidade de pessoas que usa cada um deles.

In [5]:
/*
    Nesse item contamos quantas vezes cada medicamento aparece na tabela UsoMedicamento, 
    a chave dessa tabela é a união das duas Chaves estrangeias IdPessoa e o CodMedicamento, 
    com isso garantimos que uma pessoa só aparece uma vez associdada ao mesmo remédio. 
    Logo ao contarmos o numero de vezes que o remédio aparece, também temos o número de 
    pessoas que fazem uso desse mesmo remédio.
   
*/
SELECT M.NomeMedicamento, COUNT(*) Qtd_Pessoas_Que_Usam
        FROM Medicamento M, UsoMedicamento UM 
        WHERE UM.CodMedicamento = M.Codigo 
        GROUP BY UM.CodMedicamento;



## 2) Medicamento mais usado com VIEW

* Informe o código do medicamento mais usado: fazendo uso de VIEW.

In [6]:
CREATE VIEW MaisUsado AS 
SELECT UM.CodMedicamento, M.NomeMedicamento, COUNT(*) Users
        FROM Medicamento M, UsoMedicamento UM 
        WHERE UM.CodMedicamento = M.Codigo 
        GROUP BY UM.CodMedicamento;


SELECT DISTINCT Codmedicamento 
        FROM MaisUsado 
        WHERE MaisUsado.Users = 
            SELECT MAX(Users) 
            FROM MaisUsado;


/*CREATE VIEW Contagem_Modelo AS
SELECT Modelo, COUNT(*) Numero_Taxis
       FROM taxi
       GROUP BY Modelo;

SELECT * FROM Contagem_Modelo;*/

d00732

In [ ]:
DROP TABLE IF EXISTS MaisUsado;


## 3) Medicamento mais usado sem VIEW

* Informe o código do medicamento mais usado: sem fazer uso de VIEW.

In [7]:
/*
    Encontrei 2 formas de exibir o código do medicamento, a primeira fiz vários selects aninhados, os dois mais internos 
    recuperam o valor do medicamento mais usado, no caso tem 493 usuários desse medicamento.
    Os 2 selects mais externos servem para fazer uma busca e comparar com o resultados até encontrar o de maior valor e então
    retornar o código do medicamento.
    
    Com certeza não é a melhor opção devido o tempo que leva para fazer várias contagens, mas é uma das formas de obter apenas o código.
*/
        
        SELECT CodMedicamento 
        FROM (
            SELECT UM.CodMedicamento, COUNT(*) Users  
            FROM UsoMedicamento UM 
            GROUP BY UM.CodMedicamento HAVING Users =
                SELECT MAX(Users) 
                FROM (
                    SELECT UM.CodMedicamento, COUNT(*) AS Users 
                    FROM UsoMedicamento UM 
                    GROUP BY UM.CodMedicamento)); 

        
/*
    A Segunda forma imprime não só o código do medicamento, mas também o número de usuários que fazem usso desse medicamento, 
    ainda assim não é a melhor forma, pois utiliza 2x o COUNT, o que pode ser lento.
*/      
        
        
        SELECT UM.CodMedicamento, COUNT(*) Users  
        FROM UsoMedicamento UM 
        GROUP BY UM.CodMedicamento HAVING Users =         
            SELECT MAX(Users) 
            FROM (
                SELECT UM.CodMedicamento, COUNT(*) AS Users 
                FROM UsoMedicamento UM 
                GROUP BY UM.CodMedicamento);        
        

## 4) Nome do medicamento mais usado
* Informe o nome do medicamento mais usado (uso de VIEW é opcional).

In [8]:
SELECT NomeMedicamento 
FROM Medicamento 
WHERE Codigo = 
    SELECT CodMedicamento 
    FROM (
        SELECT UM.CodMedicamento, COUNT(*) Users  
        FROM UsoMedicamento UM 
        GROUP BY UM.CodMedicamento 
        HAVING Users =
            SELECT MAX(Users) 
            FROM (
                SELECT UM.CodMedicamento, COUNT(*) AS Users 
                FROM UsoMedicamento UM 
                GROUP BY UM.CodMedicamento)); 

LISINOPRIL

## 5) Contabilizando quanto as pessoas usam de medicamento

* Informe o número médio de uso de medicamento por pessoa.

In [9]:
CREATE VIEW MediaDeUso AS
SELECT UM.IdPessoa, COUNT(*) N_medicamentos FROM UsoMedicamento UM  GROUP BY UM.IdPessoa;

In [10]:
SELECT AVG(N_Medicamentos) FROM MediaDeUso;

3

## 6) Pessoas que usam mais do que a média

* Liste o id das pessoas que usam mais medicamentos do que a média.

In [11]:
SELECT IDPessoa, N_Medicamentos 
FROM MediadeUso 
WHERE N_Medicamentos > 
    (SELECT AVG(N_Medicamentos) 
    FROM MediaDeUso) 
    ORDER BY IdPessoa;

## 7) Análise do uso de medicamento

* Considere que um médico quer fazer responder a seguinte questão: pessoas tendem a usar mais medicamentos conforme ficam mais velhas?
* Escreva uma query que ajude o médico a realizar esta análise.

In [12]:
/*
    Nessa Analise atraves do SELECT interno obtemos o numero de medicamentos por id, 
    no select externo fazemos a média do número de medicamentos que cada ide tem, e agrupamos por idade.
*/
SELECT Idade, AVG(N_Medicamentos) Media_N_Medicamentos 
FROM 
    (SELECT P.idade Idade, COUNT(*) N_Medicamentos 
    FROM Pessoa P, UsoMedicamento UM 
    WHERE P.id = UM.IdPessoa 
    GROUP BY P.id)
GROUP BY Idade;




# II. Análise de correlação em pares
## Estudo de ação cruzada entre medicamentos

* Um pesquisador quer estudar o efeito cruzado de usar dois medicamentos simultaneamente. 
* Para isso ele precisará da seguinte sequência de queries.

## 8) Medicamentos tomados em conjunto
* Construa uma VIEW com duas colunas: medicamentoA, medicamentoB
* Nessa VIEW, é criada uma tupla (medicamentoA, medicamentoB) toda vez que uma pessoa toma o medicamento A e também o medicamento B.
* Haverá repetição de tuplas, já que mais de uma pessoa pode tomar o mesmo par de medicamentos.
* Cuidado para não duplicar a mesma informação, ou seja, considerando que uma pessoa tomou o medicamentoX e o medicamentoY, cuidado para não criar duas tuplas: (medicamentoX, medicamentoY) e (medicamentoY, medicamentoX).

## 9) Total de medicamento tomados em conjunto
* Apresente o total de pessoas que toma cada par de medicamentos.

# III. Análise de correlação entre variáveis
* Consiste na análise da seguinte questão: como a alteração de uma variável afeta a outra.
## Matriz de análise
* Considere a matriz abaixo.

In [ ]:
CREATE TABLE Matriz (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  IdadeAte60 BOOLEAN,
  MaisUmAnoMedicamento BOOLEAN,
  MedicamentosAcimaMedia BOOLEAN,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade,
    FALSE, FALSE, FALSE
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

SELECT * FROM Matriz;

## 10) Atualização simples da matriz
* Utilize o comando UPDATE para atualizar a coluna IdadeAte60 da tabela colocando verdadeiro para as pessoas que têm até 60 anos de idade.

## 11) Atualização vinculada a uma segunda tabela
* Utilize o comando UPDATE para atualizar a coluna MaisUmAnoMedicamento, colocando verdadeiro para aquelas pessoas que usam pelo menos um medicamento há mais de um ano.

## 12) Atualização baseada em uma consulta complexa
* Utilize o comando UPDATE para atualizar a coluna MedicamentosAcimaMedia, colocando verdadeiro para aquelas pessoas que usam um número de medicamentos acima da média.

## 13) Que análise interessante pode ser feita?
* A partir da Matriz criada, proponha uma análise interessante.

### 13a) Descreva a seguir (em Markdown) qual a análise que você propõe.

### 13b) Escreva uma consulta em SQL que fornece informações para a análise proposta.